# Calcolo distanza scuole d'infanzia da centri urbani
## Dati disponibili
### confini comuni d'Italia da ISTAT
i dati si trovano a [questa pagina](https://www.istat.it/it/archivio/222527)<br/>
I dati sono utilizzati sono quelli dei [confini non generalizzati](https://www.istat.it/storage/cartografia/confini_amministrativi/non_generalizzati/2023/Limiti01012023.zip), in particolare quello dei confini comunali (**Com1012023_WGS84.shp**)

### elenco delle scuole da Ministero Istruzione
Da questa [risorsa](https://dati.istruzione.it/opendata/opendata/catalogo/elements1/?area=Scuole) sono presi i dati delle anagrafiche delle:
- scuole statali
- scuole paritarie
- scuole statali di Valle d'Aosta, Provincia Autonoma di Trento e Provincia Autonoma di Bolzano
- scuole paritarie di Valle d'Aosta, Provincia Autonoma di Trento e Provincia Autonoma di Bolzano<br/>
<br/>
note:
- i dati sono privi di coordinate geografiche

### elenco centri abitati da OpenStreetMap
A causa del fatto che in Italia esistono diversi comuni sparsi e del fatto di avere un punto significativo che indica il baricentro sociale di un luogo, si è optato per scaricare i dati dei centi urbani da OpenStreetMap selezionando i valori di [city](https://wiki.openstreetmap.org/wiki/Tag%3Aplace%3Dcity), [sub-urb](https://wiki.openstreetmap.org/wiki/Tag%3Aplace%3Dsuburb), [town](https://wiki.openstreetmap.org/wiki/Tag%3Aplace%3Dtown) e [village](https://wiki.openstreetmap.org/wiki/Tag%3Aplace%3Dvillage) dal tag [place](https://wiki.openstreetmap.org/wiki/Map_features#Place) <br/>
 si possono ottenere anche attraverso questa [query alle overpass-api](https://overpass-turbo.eu/s/1yku)<br/>
<br/>
note
 - la categorizzazione è delegata all'interpretazione di chi ha inserito i dati
 - il punto che rappresenta il toponimo è a discrezione di chi ha inserito i dati: in alcuni casi è il "baricentro sociale", in altri il centroide del confine comunale, in altri la piazza principale o il municipo o il campanile ...
 - ci sono diversi casi in cui è stata creata un'area. In tal caso si sceglie di utilizzare un punto significativo all'interno dell'area sulla base della [funzione di geopandas](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.representative_point.html)
   
###  grafo stradale di openstreetmap
si è preso il file pbf che si trova su [geofabrik](https://download.geofabrik.de/europe/italy-latest.osm.pbf) 

## Calcolo delle distanze
Il calcolo delle distanze avviene tramite una istanza [GraphHoper](https://www.graphhopper.com/) da eseguire sul proprio computer.<br/>
L'installazione prevede l'installazione di java.<br/>
Il software si scarica con queste istruzioni:
```bash
wget https://repo1.maven.org/maven2/com/graphhopper/graphhopper-web/7.0/graphhopper-web-7.0.jar https://raw.githubusercontent.com/graphhopper/graphhopper/7.x/config-example.yml https://download.geofabrik.de/europe/italy-latest.osm.pbf
```
l'esecuzione invece con questa istruzione
```
java -D"dw.graphhopper.datareader.file=italy-latest.osm.pbf" -jar graphhopper*.jar server config-example.yml
``````
L'istruzione *-D"dw.graphhopper.datareader.file=italy-latest.osm.pbf"* può essere ignorata qualora venga modificato il file *config-example.yml* con il giusto riferimento al file.

## Geocoding
Per arricchire le informazioni sulle posizioni delle scuole d'infanzia viene utilizzato il servizio di [geocoding di ESRI ArcGIS](https://developers.arcgis.com/documentation/mapping-apis-and-services/geocoding/geocode-addresses/) sempre attraverso [geopandas](https://geopandas.org/en/stable/docs/reference/api/geopandas.tools.geocode.html)<br/> 
Il geocoding comunque spesso restituisce valori che vanno verificati i corretti.<br/>
<br/>
Alcuni consigli per effettuare delle verifiche:
- il punto si trova all'interno del comune interessato?
- il punto si trova vicino alla via dichiarata? (controllo con uso di reverse-geocoding?)<br/>
Ulteriori verifiche possono essere fatte con ricerche mirate online e attraverso la verifica visita attraverso strumenti come Google Street View<br/>
<br/>
note<br/>
- fare molta attenzione al numero di chiamate al secondo

# Procedura
Lo script jupiter notebook [calcolo_distanze_centri_abitati_scuole_infanzia.ipynb](calcolo_distanze_centri_abitati_scuole_infanzia.ipynb) carica prima i dati cercando nella sottodirectory "data".<br/>
Da qui fa una serie di operazioni di pulizia/integrazione dei dati che archivia sempre in "data" in modo che, ad un secondo avvio dello script, non debba rifare quelle operazioni.<br/>
La sezione che si occupa del calcolo delle distanze dalle scuole di infanzia ai centri abitati filtra per una provincia (allo stato attuale è stata inserita Bergamo).<br/>
Da qui filtra quindi i dati delle scuole e dei confini comunali della provincia scelta.<br/>
Il calcolo delle distanze avviene prendendo poi un comune alla volta, individuando i centri abitati al suo interno e quelli dei comuni confinanti (questo per ridurre i tempi di attesa dei calcoli) - sono quindi *esclusi* i comuni confinanti di altra provincia o regione.<br/>
Allo stesso modo individua le scuole d'infanzia (statali o paritarie) contenute nei comuni individuati e avvia le chiamate per il calcolo delle distanze interrogando l'instanza GraphHoper avviata sul proprio computer.<br/>
Le distanze sono archiviate in metri ed i tempi di pecorrenza in millesecondi.<br/>
La tabella viene generata in "data" con il nome di "distanze_scuoleinfanzia_centriabitati_*Provincia*.csv" dove *Provincia* corrisponde al nome della provincia scelta.<br/>
Nel file vengono poi riportati:
- nome della scuola
- indirizzo
- coordinate scuola
- codice identificativo della scuola
- comune
- codice comune istat
- nome del centro abitato
- comune dove si trova il centro abitato
- codice istat del comune
- coordinate del centro abitato
- tempo di percorrenza dalla scuola al centro abitato in millesecondi utilizzando l'auto
- distanza percorsa dalla scuola al centro abitato in metri utilizzando l'auto


In [126]:
import geopandas as gpd
import time
import pandas as pd
import os
import requests
import warnings
import requests
import urllib3
import io
requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS = 'ALL:@SECLEVEL=1'
warnings.filterwarnings("ignore", category=DeprecationWarning) 
pd.set_option('mode.chained_assignment', None)

## Caricamento dei dati

In [127]:
comuni_italiani = gpd.read_file("data" + os.sep + "Com01012023_WGS84.shp", encoding='utf-8')

In [128]:
res = requests.get("https://dati.istruzione.it/opendata/opendata/catalogo/elements1/SCUANAGRAFESTAT20232420230901.csv",verify=False, timeout=120)
scuole_italiane_statali = pd.read_csv(io.BytesIO(res.content), sep=',')
res = requests.get("https://dati.istruzione.it/opendata/opendata/catalogo/elements1/SCUANAGRAFEPAR20232420230901.csv",verify=False, timeout=120)
scuole_italiane_paritarie = pd.read_csv(io.BytesIO(res.content), sep=',')
res = requests.get("https://dati.istruzione.it/opendata/opendata/catalogo/elements1/SCUANAAUTSTAT20232420230901.csv",verify=False, timeout=120)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dati.istruzione.it'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dati.istruzione.it'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dati.istruzione.it'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.i

In [129]:
res = requests.get("https://dati.istruzione.it/opendata/opendata/catalogo/elements1/SCUANAAUTSTAT20232420230901.csv")
scuole_province_autonome_statali = pd.read_csv(io.BytesIO(res.content), sep=',')
res = requests.get("http://dati.istruzione.it/opendata/opendata/catalogo/elements1/SCUANAAUTPAR20232420230901.csv")
scuole_province_autonome_paritarie = pd.read_csv(io.BytesIO(res.content), sep=',')

In [130]:
if os.path.exists("data" + os.path.sep + "centri_abitati_italia_osmistat.gpkg"):
    centri_abitati_italia_osmistat = gpd.read_file("data" + os.path.sep + "centri_abitati_italia_osmistat.gpkg")
else:
    centri_abitati_italia = gpd.read_file("data" + os.sep + "centri_abitati_italia.geojson")

## Rielaborazione centri abitati

In [131]:
if os.path.exists("data" + os.path.sep + "centri_abitati_italia_osmistat.gpkg") == False:
    centri_abitati_italia.geometry = centri_abitati_italia.geometry.representative_point()
    mask = ~centri_abitati_italia['name'].isna()
    centri_abitati_italia = centri_abitati_italia[mask]
    nomi_centri_abitati = centri_abitati_italia[["name","place","geometry"]]
    nomi_centri_abitati_istat = gpd.sjoin(nomi_centri_abitati,
                                comuni_italiani.to_crs(epsg=4326), 
                                how='inner', predicate='within', lsuffix='osm_', rsuffix='istat_')
    centri_abitati_italia_osmistat = nomi_centri_abitati_istat.reset_index()[['COD_PROV','PRO_COM','PRO_COM_T','COMUNE',"name","place","geometry"]]
    new_columns = {}
    for column in centri_abitati_italia_osmistat.columns:
        new_columns[column] = column.lower()
    centri_abitati_italia_osmistat.rename(columns=new_columns,inplace=True)
    centri_abitati_italia_osmistat.to_file("data" + os.path.sep + "centri_abitati_italia_osmistat.gpkg",driver="GPKG")

## Scuole dell' infanzia

In [132]:
scuole_italiane = pd.concat([
    scuole_italiane_statali[scuole_italiane_paritarie.columns],
    scuole_italiane_paritarie,
    scuole_province_autonome_paritarie,
    scuole_province_autonome_statali[scuole_province_autonome_paritarie.columns]])

In [133]:
scuole_infanzia_italiane = scuole_italiane[scuole_italiane.DESCRIZIONETIPOLOGIAGRADOISTRUZIONESCUOLA.isin(['SCUOLA INFANZIA','SCUOLA INFANZIA NON STATALE'])]

In [134]:
scuole_infanzia_italiane[scuole_infanzia_italiane.PROVINCIA=="TRENTO"]

,ANNOSCOLASTICO,AREAGEOGRAFICA,REGIONE,PROVINCIA,CODICESCUOLA,DENOMINAZIONESCUOLA,INDIRIZZOSCUOLA,CAPSCUOLA,CODICECOMUNESCUOLA,DESCRIZIONECOMUNE,DESCRIZIONETIPOLOGIAGRADOISTRUZIONESCUOLA,INDIRIZZOEMAILSCUOLA,INDIRIZZOPECSCUOLA,SITOWEBSCUOLA


## Calcolo su Provincia di Bergamo

In [135]:
provincia = 'Lecco'

### geometrie

In [136]:
codice_provincia = comuni_italiani[comuni_italiani.COMUNE == provincia].COD_PROV.values[0]
comuni_provincia = comuni_italiani[comuni_italiani.COD_PROV == codice_provincia]
confine_provincia = comuni_provincia.to_crs(epsg=4326).dissolve().geometry.values[0]

### Scuole infanzia della provincia e georeferenziazione

In [137]:
scuole_italiane_provincia  = scuole_infanzia_italiane[scuole_infanzia_italiane.PROVINCIA == provincia.upper()]

In [138]:
geoscuolefilename = "geo_scuole_infanzia_" + provincia.lower().replace(" ","_").replace("'","_") + ".parquet"

In [139]:
if os.path.isfile("data" + os.sep + geoscuolefilename) == False:
    data = []
    for idx, row in scuole_italiane_provincia.iterrows():
        datarow = {}
        codice = row['CODICESCUOLA']
        q = row['INDIRIZZOSCUOLA'] + ", " + row['CAPSCUOLA'] + " " + row['DESCRIZIONECOMUNE'] + ", " + row['PROVINCIA'] + ", Italia"
        point = gpd.tools.geocode(q, provider="arcgis")
        lon = str(point.geometry.x.values[0])
        lat = str(point.geometry.y.values[0])
        datarow['codice_scuola'] = codice
        datarow['lon'] = lon
        datarow['lat'] = lat
        datarow['address'] = row['INDIRIZZOSCUOLA']
        datarow['city'] = row['DESCRIZIONECOMUNE']
        datarow['name'] = row['DENOMINAZIONESCUOLA']
        data.append(datarow)
        time.sleep(5)
    geodata_scuole = pd.DataFrame(data)
    geodata_scuole.to_parquet("data" + os.sep + geoscuolefilename)
else:
    geodata_scuole = pd.read_parquet("data" + os.sep + geoscuolefilename)

In [140]:
geodata_scuole = gpd.GeoDataFrame(
    geodata_scuole,
    crs='EPSG:4326',
    geometry=gpd.points_from_xy(geodata_scuole.lon, geodata_scuole.lat))


### Centri abitati della provincia

In [141]:
centri_abitati_provincia = centri_abitati_italia_osmistat[centri_abitati_italia_osmistat.within(confine_provincia)]

## Calcolo delle distanze
questo calcolo richiede che sia in esecuzione graphhopper sulla macchina locale<br/>
inoltre per ottimizzare le richieste calcola solo i possibili spostamenti fra comuni confinanti

In [142]:
url_route =  "http://localhost:8989/route"

In [143]:
data_scuole = []
for idx, row in geodata_scuole.iterrows():
    scuola = {}
    lat = row['lat']
    lon = row['lon']
    scuola["codice_scuola"]= row['codice_scuola']
    scuola['lat'] = lat
    scuola['lon'] = lon
    scuola['school_address'] = row['address']
    scuola['school_name'] = row['name']
    scuola['school_place'] = row['city']
    data_scuole.append(scuola)

In [144]:
distanzescuolefilename = "distanze_scuoleinfanzia_centriabitati_" + provincia.lower().replace(" ","_").replace("'","_") + ".csv"

## Ciclo su comuni confinanti

In [145]:
def getdistance(scuole,centri):
    rdata = []
    for idx, centro_urbano in centri.iterrows():
        for sidc, scuola in scuole.iterrows():
            data_distance = {}
            data_distance['from_pro_com'] = scuola['PRO_COM']
            data_distance['from_comune'] = scuola['COMUNE'] 
            data_distance["from_id_school"] = scuola['codice_scuola']
            data_distance['from_name'] = scuola['name']
            data_distance['from_address'] = scuola['address']
            data_distance['latitude_school'] = scuola['lat']
            data_distance['longitude_school'] = scuola['lon']
            post_json = {
                "points": [[float(scuola['lon']),float(scuola['lat'])],[centro_urbano.geometry.x, centro_urbano.geometry.y]],
                "profile":"car",
                "elevation":"false",
                "debug":"false",
                "instructions":"false",
                "locale":"en_US",
                "optimize":"false",
                "points_encoded":"true",
                "snap_preventions": ["ferry"],
                "details":["road_class","road_environment","max_speed","average_speed"]}
            r = requests.post(url = url_route,json=post_json)
            data = r.json()
            data_distance["to"] = centro_urbano.name
            data_distance["latitude_to"] = centro_urbano.geometry.y
            data_distance["longitude_to"] = centro_urbano.geometry.x
            data_distance["to_kind"] = centro_urbano.place
            data_distance["to_comune"] = centro_urbano.comune
            data_distance["to_pro_com"] = centro_urbano.pro_com
            if "message" in data.keys():
                #caso in cui il punto non è sul grafo stradale
                #possibile errore di geocoding che restituisce il centro del paese
                distance = -1
                travel_time = -1
            else:
                distance = data["paths"][0]['distance']
                travel_time = data["paths"][0]['time']
            data_distance["distance"] = distance
            data_distance["travel_time"] = travel_time
            rdata.append(data_distance)
    return(rdata)

In [146]:
if os.path.exists("data" + os.sep + distanzescuolefilename) == False:
    distanze_scuole_centri = []
    for idx,row in comuni_provincia.iterrows():
        g=row['geometry']
        comuni_limitrofi = comuni_provincia[comuni_provincia.touches(g)]
        scuole_comuni_limtrofi = gpd.sjoin(geodata_scuole,comuni_limitrofi.to_crs(geodata_scuole.crs),predicate="within")
        scuole_comuni_limtrofi = scuole_comuni_limtrofi[['codice_scuola','address','name','lon','lat','geometry','PRO_COM','COMUNE']]
        scuole_comune = geodata_scuole[geodata_scuole.to_crs(comuni_provincia.crs).within(row['geometry'])]
        scuole_comune['PRO_COM'] = row['PRO_COM']
        scuole_comune['COMUNE'] = row['COMUNE']
        scuole_vicino_comune = pd.concat([scuole_comuni_limtrofi,scuole_comune])
        centri_abitati_comune = centri_abitati_italia_osmistat[centri_abitati_italia_osmistat.to_crs(comuni_provincia.crs).within(g)]
        if len(distanze_scuole_centri) == 0:
            distanze_scuole_centri = getdistance(scuole_vicino_comune,centri_abitati_comune)
        else:
            distanze_scuole_centri = getdistance(scuole_vicino_comune,centri_abitati_comune) + distanze_scuole_centri
    distanze = pd.DataFrame(distanze_scuole_centri) 
    distanze.to_csv("data" + os.sep + distanzescuolefilename,index=False)
else:
    distanze = pd.read_csv("data" + os.sep + distanzescuolefilename)   